In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My\ Drive/human-tracker

/content/drive/My Drive/human-tracker


In [5]:

# Code adapted from Tensorflow Object Detection Framework
# Code modified for Google Colab settings.
# https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
# Tensorflow Object Detection Detector

# Edmond Tsoi for EECS 442 Final Project, Winter 2020
# Faster RCNN

# Dependencies: TensorFlow v2.1.0, Python3.7, OpenCV 4.2

import numpy as np
import os 
import tensorflow as tf
import cv2
import time
import pickle
from google.colab.patches import cv2_imshow
# Supress Warning message
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

print("TensorFlow version: {}".format(tf.__version__))
print("OpenCV version: {}".format(cv2.__version__))

class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.io.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.compat.v1.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Frame Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()

if __name__ == "__main__":
    # TODO: Modify the model path 
    model_path = '/content/drive/My Drive/human-tracker/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.9

    # Walking video footage adapted from: https://www.youtube.com/watch?v=QyUi149TcPQ
    # TODO: Modify the input video path (You can choose any video as you wish.) 
    # cap = cv2.VideoCapture('/Users/kawaitsoi/Desktop/ts1/human-tracker-ts1/walking.mp4')
    cap = cv2.VideoCapture('/content/drive/My Drive/human-tracker/people_walking.mp4')
    pts = []
    count = 0

    # Adapt for google colab way of displaying video.
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    # make sure resolution matches frame resolution.
    out = cv2.VideoWriter('output-test-temp.mp4', fourcc, 20.0,(1280, 720))
    outer_box = []
    while True:
        count += 1
        r, img = cap.read()
        if not r:
          break
        img = cv2.resize(img, (1280, 720))
        boxes, scores, classes, num = odapi.processFrame(img)
        # Visualization of the results of a detection.
        for i in range(len(boxes)):
            # Class 1 represents human.
            if classes[i] == 1 and scores[i] > threshold:
                box = boxes[i]
                outer_box.append(box)
                # cv2.fill
                cv2.putText(img, "Person", (box[1], box[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)
                # center_coordinates = ((box[3] - box[1]) // 2, (box[2] - box[0]) // 2)
                x = [box[1], box[3]]
                y = [box[0], box[2]]
                center_coordinates = (sum(x) // 2, max(y)-5)
                pts.append(center_coordinates)
                for pt in pts:
                    cv2.circle(img, pt, 2, (0, 0, 255), 1)
                # if count % 50 == 0:
                #     cv2.findContours(img, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                    # cv2.drawContours(img, )

            # Class 3 represents car.
            # if classes[i] == 3 and scores[i] > threshold:
            #     box = boxes[i]
            #     cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(0,128,0),2)
            
        # cv2_imshow(img)
        out.write(img)
        # key = cv2.waitKey(1)
        # if key & 0xFF == ord('q'):
        #     break
    file = open('important', 'wb')
    pickle.dump(outer_box, file)
    file.close()
    out.release()


TensorFlow version: 2.2.0-rc3
OpenCV version: 4.1.2
Frame Time: 3.084533929824829
Frame Time: 0.07566356658935547
Frame Time: 0.07577061653137207
Frame Time: 0.07624244689941406
Frame Time: 0.08465743064880371
Frame Time: 0.08556342124938965
Frame Time: 0.07777953147888184
Frame Time: 0.07564544677734375
Frame Time: 0.07236075401306152
Frame Time: 0.07339954376220703
Frame Time: 0.06936907768249512
Frame Time: 0.0765542984008789
Frame Time: 0.07533907890319824
Frame Time: 0.07247304916381836
Frame Time: 0.0749976634979248
Frame Time: 0.07364916801452637
Frame Time: 0.07791471481323242
Frame Time: 0.07554769515991211
Frame Time: 0.06810498237609863
Frame Time: 0.07542610168457031
Frame Time: 0.07123565673828125
Frame Time: 0.07464480400085449
Frame Time: 0.07358860969543457
Frame Time: 0.0763094425201416
Frame Time: 0.07378315925598145
Frame Time: 0.07413411140441895
Frame Time: 0.07230019569396973
Frame Time: 0.07663297653198242
Frame Time: 0.07756996154785156
Frame Time: 0.07906842231

In [19]:
ls

faster_rcnn_inception_v2_coco_2018_01_28/  ssd_inception_v2_coco_11_06_2017/
output-test.mp4                            walking.mp4
